In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
import openai
openai.api_key = "sk-xxx"

In [4]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
andrew_gina_docs = SimpleDirectoryReader(input_files=["./assets/AndrewHuberman/sleep/115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt"], filename_as_id=True).load_data()
vector_index = VectorStoreIndex.from_documents(andrew_gina_docs)

In [6]:
vector_engine = vector_index.as_chat_engine()

In [5]:
from llama_index import ListIndex, ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=1024)
nodes = service_context.node_parser.get_nodes_from_documents(andrew_gina_docs)
list_index = ListIndex(nodes)

In [7]:
list_query_engine = list_index.as_query_engine(
    response_mode = "tree_summarize", use_async = True
)

In [8]:
from llama_index.tools.query_engine import QueryEngineTool

list_tool = QueryEngineTool.from_defaults(
    query_engine = list_query_engine,
    description="Useful for summarisation of the podcast about sleep and memory with dr. Gina Poe"
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_engine,
    description="Useful fro retrieving of specific content about sleep and memory in the podcast topic"
)

In [9]:
from llama_index.selectors.pydantic_selectors import PydanticSingleSelector
from llama_index.query_engine.router_query_engine import RouterQueryEngine

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool
    ]
)

In [27]:
from llama_index.selectors.llm_selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool
    ] 
)

In [23]:
openai.log = "none"

In [29]:
from llama_index.response.pprint_utils import pprint_response
response = query_engine.query("Give me a quick summary of the Andrew Huberman podcast with dr. Gina Poe.")
pprint_response(response, show_source=True)

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: Useful for summarisation of the podcast about sleep and memory with dr. Gina Poe.
Selecting query engine 0: Useful for summarisation of the podcast about sleep and memory with dr. Gina Poe.
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4733 request_id=5f6f6bc8b6a343747ba76c35c6ae7ef0 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4733 request_id=5f6f6bc8b6a343747ba76c35c6ae7ef0 response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=5057 request_id=844ababe245e7c2a515898e390f04d08 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=5057 request_id=844ababe245e7c2a515898e390f04d08 response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/c

In [11]:
import nest_asyncio
nest_asyncio.apply()